<a href="https://colab.research.google.com/github/ZeeshanMalik1020/Test-Project/blob/main/Rag_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain pinecone-client google-generativeai openai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.9 MB/s eta 0:00:00


In [ ]:
%pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:

from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec


pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "rag-project"  # change if desired

pc.create_index(
   name=index_name,
   dimension=768,
   metric="cosine",
   spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


index = pc.Index(index_name)

In [ ]:
pip install langchain_google_genai

  Using cached langchain_google_genai-2.0.8-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.7 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vector = embeddings.embed_query("Hello world")

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
# data save
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [ ]:
len(documents)

10

In [ ]:
from uuid import uuid4
uuid4()

UUID('5d4eabe2-c264-4946-bea4-9627e1aee73a')

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['f13abbf1-e06f-401c-a6f9-4d3cb22aa88b',
 'd5aae8a8-3c1b-4e73-ae92-4bb25574cb21',
 '8c5538fc-3fe8-4f7e-a5ab-84fbe6eeac22',
 '0d11cd80-9e62-4940-af61-727615d46745',
 'a54ac669-a106-4aa5-ab7b-6f5989ee462e',
 '6bc42fe5-8f3e-42c9-8d74-d18d5f79b9fd',
 'f2d3ece6-002f-479d-a5f7-456000d83094',
 'c774fd79-736e-42c1-a578-12d61960e355',
 'ea90608d-d0e7-4977-8426-b91171eef836',
 '1fee7097-ea7c-46a9-97df-daa54062717c']

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
def answer_to_user(query: str):
    results = vector_store.similarity_search(query, k=2)
    final_answer = model(results, query)
    return final_answer
